In [18]:
import math
from scipy.stats import norm
from scipy.optimize import fsolve

In [5]:
S0 = 58
K = 60
r = 0.02
q = 0.01
T = 9/12


In [17]:
def blackScholesVanilla(K , T , S , v , q , r , CP ):
    phi = 1 if CP == 'C' else -1
    dp = (1/(v*math.sqrt(T)))*(math.log(S/K) + ((r - q + (v**2)/2)*T) )
    dm = dp - (v*math.sqrt(T))
    vBS = phi*((norm.cdf(phi*dp)*(S*math.exp(-q*T))) - ((norm.cdf(phi*dm))*(K*math.exp(-r*T))))
    return vBS

In [23]:
def calculateIV(v , K , T , S , q , r , CP , Vm):
    
    modelV = blackScholesVanilla(K , T , S , v , q , r , CP )
    eq = modelV - Vm
    return eq

In [24]:
IV = fsolve(calculateIV , x0 = 0.25 , args =( K , T , S0 , q, r , 'P', 6.36))

In [25]:
print(IV)

[0.27633884]


In [2]:
def binomialTreePricer(S , K , v, r , q , T , N):
    deltaT = T/N
    u = math.exp(v*math.sqrt(deltaT))
    d = 1/u
    f = math.exp((r-q)*(deltaT))
    df = math.exp(-r*deltaT)
    pU = (f - d)/(u-d)
    pD = 1 - pU
    V_N = []
    for i in range(N+1):
        V_N.append(max(K - S*(u**(N-i))*(d**(i)),0))
    
    for j in range(N-1,-1,-1):
        V = []
        for i in range(j+1):
            p = df*((pU*V_N[i]) + (pD*V_N[i+1]))
            V.append(p)
        V_N = V
    
    return V_N[0]

    

In [26]:
tol = 1
N = 10
tol = 1e-4
v = IV[0]
prevV = binomialTreePricer(S0 , K , v , r , q , T , N)
N = 2*N
currV = binomialTreePricer(S0 , K , v , r , q , T , N)

print(prevV ,currV)
while abs(currV - prevV) > tol:
    
    prevV = currV
    N = 2*N
    currV = binomialTreePricer(S0 , K , 0.27 , r , q , T , N)
    
    print(N , prevV,currV)

print(N , currV)
    
    

6.409608247718533 6.408727248207288
40 6.408727248207288 6.266540548039494
80 6.266540548039494 6.24758557881732
160 6.24758557881732 6.230544710778612
320 6.230544710778612 6.236662650205346
640 6.236662650205346 6.234128261588947
1280 6.234128261588947 6.234890053993187
2560 6.234890053993187 6.23432870866447
5120 6.23432870866447 6.2339828808779725
10240 6.2339828808779725 6.234030983382823
10240 6.234030983382823


In [36]:
N_fixed = 10240
print(N_fixed)

10240


In [37]:
Vm = 6.36

In [38]:
#Secant Implementation
iv0 = 0.05
iv1 = 1
tol = 1e-4
while abs(iv0 - iv1) > tol:
    f1 = binomialTreePricer(S0 , K , iv1 , r , q , T , N_fixed)
    f0 = binomialTreePricer(S0 , K , iv0 , r , q , T , N_fixed)
    iv2 = iv1 - (((f1-Vm)*(iv1-iv0))/(f1 - f0))
    iv0 = iv1
    iv1 = iv2
    
    print(iv0 , f0 , iv1 , f1)

1 1.9593947445036421 0.27768491069666446 20.32062465618257
0.27768491069666446 20.32062465618257 0.2762978132219829 6.386757895111223
0.2762978132219829 6.386757895111223 0.276338165616187 6.359198255744719
